In [1]:
import model as m
import numpy as np

import os

os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.set_logical_device_configuration(
                gpu,
                [tf.config.LogicalDeviceConfiguration(memory_limit=256)])

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')

    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2022-03-22 12:35:52.206200: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-03-22 12:35:52.206367: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 256 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


In [3]:
model= m.NN_Model()
model.get_data("laser_log/laser_log.csv")

beta=[0.2,0.8]
drop=[0.3]
epochs=12
batch_size=16

group_epochs=2

results=[]

for i in beta:
    for j in drop:
        #with sess.as_default():
            model.model_define(drop_rate=j)
            model.set_beta(i)
            model.model_compile("adam",model.pos_loss,['mean_absolute_error'])
            for step in range(epochs//group_epochs):
                history = model.model_run(group_epochs,batch_size, verbose=1)
                pos_mse,rot_mse,pos_mae,rot_mae = model.get_error(verbose=1)
                results.append({"beta":i,"drop-out":j,"epochs":(step+1)*group_epochs,"history":history,"position squared error":pos_mse, "rotation squared error":rot_mse, "position absolute error":pos_mae, "rotation absolute error":rot_mae})  

Epoch 1/2


2022-03-22 12:37:27.321381: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsync failed to allocate 188743680 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 197459968/2095448064
2022-03-22 12:37:27.321426: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:293] Stats: Limit:                       268435456
InUse:                       959058036
MaxInUse:                   1145997900
NumAllocs:                       56998
MaxAllocSize:                188743680
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2022-03-22 12:37:27.321442: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:56] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2022-03-22 12:37:27.321450: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator

ResourceExhaustedError: in user code:

    File "/home/ait_guest/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/ait_guest/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ait_guest/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/ait_guest/.local/lib/python3.9/site-packages/keras/engine/training.py", line 863, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/ait_guest/.local/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 532, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "/home/ait_guest/.local/lib/python3.9/site-packages/keras/optimizer_v

In [3]:
with open('results23k.csv', 'w') as f:
    f.write("beta;dropout;epochs;pos_mse;pos_mae;rot_mse;rot_mae\n")
    for item in results:
        f.write("{};{};{};{};{};{};{}\n".format(item["beta"],item["drop-out"],item["epochs"],item["position squared error"],item["position absolute error"],item["rotation squared error"],item["rotation absolute error"]))